In [1]:
from konlpy.tag import Mecab
import pandas as pd
import numpy as np
import time
import re

from sklearn.feature_extraction.text import CountVectorizer

### 파일 불러오기

In [2]:
reviews = pd.read_csv('./골목식당_reviews.csv', encoding='utf-8')
reviews.drop('Unnamed: 0', axis=1, inplace=True)
reviews.head(2)

,storeid,Content
0,200000,"가야가야서울특별시 서대문구 이화여대5길 7-3전화상세보기&lt;a onclick=""..."
1,200000,위치가야가야오랜만에 이대에 갈 일이 있어 (이대다니던 여자는 아니구염 ㅋㅋㅋ)제가 ...


In [3]:
def names_review(reviews_df, column_name):
    
    stop_words = set(['은','는','이','가','하','아','것','들','의','있','그','되','수','보','주','등','한'
                      ,'나','같','우리','때','년','지','오','말','일','그렇','위하','때문','그것'
                      ,'두','말하','알','그러나','받','일','또','그런','문제','그리고','크','중','나오','가지'
                      ,'시키','지금','생각하','그러','속','하나','집','살','적','월','데','자신','안','어떤'
                      ,'내','경우','명','생각','시간','그녀','다시','이런','앞','보이','번','나','다른','어떻','전','들'
                      ,'사실','이렇','점','싶','말','정도','좀','원','통하','소리','놓'
                      ,'error','으로','잇님','잇님들'])
    
    mecab = Mecab()
    vect = CountVectorizer(
                            tokenizer=mecab.morphs
                            , stop_words=stop_words
                          ) 
    
    hangeul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    
    bow_names_list = []
    for i in range(len(reviews_df)):
        reviews_df.reset_index(drop=True,inplace=True )
        try:                
            tmp = [hangeul.sub('',reviews_df.loc[i, column_name])]
            vect.fit(tmp)
            bow_names = vect.get_feature_names()
            bow_names_list.append(bow_names)
        
        except:
            bow_names_list.append('error')
        
    return bow_names_list

### 형태소 분석 실행

In [4]:
%%time

names = names_review(reviews, "Content")
reviews['mecab'] = names

/anaconda3/envs/text_anal/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['님', '잇'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


CPU times: user 3.25 s, sys: 117 ms, total: 3.37 s
Wall time: 3.61 s


In [6]:
reviews.head(2)

,storeid,Content,mecab
0,200000,"가야가야서울특별시 서대문구 이화여대5길 7-3전화상세보기&lt;a onclick=""...","[ㅠ시원한, 고, 기, 길, 냉, 너무, 던, 도, 뜨끈, 라멘, 맛있, 매콤, 배..."
1,200000,위치가야가야오랜만에 이대에 갈 일이 있어 (이대다니던 여자는 아니구염 ㅋㅋㅋ)제가 ...,"[ㄷ, ㅋ, ㅋㅋ, ㅋㅋㅋ, ㅍㅎㅎㅎㅎ가야가야서울특별시, ㅗ갑, ㅠ, 갈, 감성, ..."


In [ ]:
reviews.to_csv('reviews_mecab_골목식당.csv', encoding='utf-8')